In [4]:
!pip install huggingface_hub --upgrade -q



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
!git config --global user.email "efegor20082008@gmail.com"
!git config --global user.name "Egor"

In [6]:
!git add Базовая_для_LLM_Generation.ipynb
!git commit -m "Первый коммит"





[main (root-commit) 1414625] Первый коммит
 1 file changed, 1912 insertions(+)
 create mode 100644 "\320\221\320\260\320\267\320\276\320\262\320\260\321\217_\320\264\320\273\321\217_LLM_Generation.ipynb"


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
!git branch -a


* main


## Если скачивать локально

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="Qwen/Qwen3-4B-Instruct-2507", local_dir="./Qwen3-4B-Instruct-2507")
model = AutoModelForCausalLM.from_pretrained(
   './gemma-3-12b-it',
    torch_dtype="auto",
    device_map="auto",
    local_files_only=True,
    trust_remote_code=True

).eval()
tokenizer = AutoTokenizer.from_pretrained('./Qwen3-4B-Instruct-2507')
# ЕСЛИ ЛОКАЛЬОНО ЧЕРЕЗ HF_HUB

## Если нелокально

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
   'Qwen/Qwen3-4B-Instruct-2507',
    torch_dtype="auto",
    device_map="auto",

).eval()
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-4B-Instruct-2507')
# ЕСЛИ НЕЛОКАЛЬНО!

## ЕСЛИ AUTOAWQ, то не забудь pip install awq (не везде работает, например в kaggle не работает)

In [ ]:
!pip install autoawq -q
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model = AutoModelForCausalLM.from_pretrained(
   'Qwen/Qwen2.5-7B-Instruct-AWQ',
    torch_dtype="auto",
    device_map="auto",
).eval()
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-7B-Instruct-AWQ')

## Если gguf, то используй этот код:

In [2]:
!ls -lh


total 112526688
-rw-r--r--@     1 quezaltendes  staff     0B Mar 25  2025 #include <iostream>.cpp
-rw-r--r--@     1 quezaltendes  staff    42K Sep  3 23:57 -10155.txt
-rw-------@     1 quezaltendes  staff    13K Dec  2  2024 -2147483648_-210738.jpg
-rw-------@     1 quezaltendes  staff    11K Oct 20 00:39 -2147483648_-220010.jpg
-rw-r--r--@     1 quezaltendes  staff    26K Sep  3 23:56 -3960.csv
-rw-r--r--@     1 quezaltendes  staff    26K Sep  3 23:56 -3960.txt
-rw-r--r--@     1 quezaltendes  staff   8.5K Aug 27 03:04 001Na93Ily8i0zphqtgfnj6050050jrc02.jpg
-rw-------@     1 quezaltendes  staff   286M Oct 28 14:16 1.pdf
-rw-r--r--@     1 quezaltendes  staff    66K Mar 16  2025 10 класс.pdf
-rw-------@     1 quezaltendes  staff   570K Dec 13  2024 10-11 класс. БИБН (1).pdf
-rw-------@     1 quezaltendes  staff   570K Dec 13  2024 10-11 класс. БИБН.pdf
-rw-r--r--@     1 quezaltendes  staff    73K Jul  7 16:50 1492098383257027447.jpg
-rw-r--r--@     1 quezaltendes  staff   105K Oct  5 19:

In [3]:
!pip install llama-cpp-python
!wget https://huggingface.co/Qwen/Qwen3-8B-GGUF/resolve/main/qwen3-8b.Q4_K_M.gguf
from llama_cpp import Llama
import os
os.make
model = Llama(model_path="qwen3-8b.Q4_K_M.gguf")

output = model("Привет! Расскажи шутку о нейросетях.", max_tokens=128)
print(output["choices"][0]["text"])



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
zsh:1: command not found: wget


AttributeError: module 'os' has no attribute 'make'

## Пишем промпт/получаем ответ

In [ ]:
prompt = "what's 2 + 2?"
messages = {'role': 'user', 'content': prompt}
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors='pt').to(model.device)



In [ ]:
with torch.no_grad():
    gen_ids = model.generate(
        inputs=**model_inputs,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        top_k=50
    )

In [ ]:
# 1. Жадный поиск (детерминированный)
model.generate(do_sample=False)
# Результат: "Погода сегодня хорошая."

# 2. Случайная выборка с температурой
model.generate(do_sample=True, temperature=0.7)
# Результат: "Погода сегодня прекрасная." (вариант 1)
# Результат: "Погода сегодня замечательная." (вариант 2)

# 3. Только top-p выборка
model.generate(do_sample=True, top_p=0.9)
# Исключает маловероятные варианты

# 4. Комбинированная стратегия
model.generate(do_sample=True, temperature=0.5, top_p=0.9, top_k=50)
# Баланс между качеством и разнообразием

In [ ]:
output_ids = gen_ids[0][len(model_inputs.input_ids[0]):].tolist() 
answer = tokenizer.decode(output_ids, skip_special_tokens=True)


## Экстрактор ответа пишешь сам, но посмотри на выводы моделй в конце

In [ ]:
# скорее всего делай что-то такое
last_string = answer.split('\n')
# было и такое:
another = []
for q in answers_by_model:
    q = q.split('\n')[-1].split(':')[-1].replace(' ', '')
    another.append(q)


NameError: name 'answer' is not defined